In [1]:
from pyspark.sql import SparkSession, functions as f

In [2]:
spark = (
    SparkSession
    .builder
    .appName("Hands-on-3")
    .master("local[*]")
    .getOrCreate()
)

### Setting up what we had done in the previous hands-on

In [3]:
df_ratings = (
    spark
    .read
    .csv(
        path="../../data-sets/ml-latest/ratings.csv",
        sep=",",
        encoding="UTF-8",
        header=True,
        quote='"',
        schema="userId INT, movieId INT, rating DOUBLE, timestamp INT",
    )
    .withColumnRenamed("timestamp", "timestamp_unix")
    .withColumn("timestamp", f.to_timestamp(f.from_unixtime("timestamp_unix")))
    .drop("timestamp_unix")
)

In [4]:
df_ratings.show(n=5)
df_ratings.printSchema()

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|     1|    307|   3.5|2009-10-28 02:30:21|
|     1|    481|   3.5|2009-10-28 02:34:16|
|     1|   1091|   1.5|2009-10-28 02:34:31|
|     1|   1257|   4.5|2009-10-28 02:34:20|
|     1|   1449|   4.5|2009-10-28 02:31:04|
+------+-------+------+-------------------+
only showing top 5 rows

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [5]:
# As we anyway dropping timestamp_unix, we can directly apply the trasformations on timestamp column

df_ratings = (
    spark
    .read
    .csv(
        path="../../data-sets/ml-latest/ratings.csv",
        sep=",",
        encoding="UTF-8",
        header=True,
        quote='"',
        schema="userId INT, movieId INT, rating DOUBLE, timestamp INT",
    )
    .withColumn("timestamp", f.to_timestamp(f.from_unixtime("timestamp")))
)

In [6]:
df_ratings.show(n=5)
df_ratings.printSchema()

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|     1|    307|   3.5|2009-10-28 02:30:21|
|     1|    481|   3.5|2009-10-28 02:34:16|
|     1|   1091|   1.5|2009-10-28 02:34:31|
|     1|   1257|   4.5|2009-10-28 02:34:20|
|     1|   1449|   4.5|2009-10-28 02:31:04|
+------+-------+------+-------------------+
only showing top 5 rows

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [7]:
# We are now going to load movies.csv for furthur analysis

df_movies = (
    spark
    .read
    .csv(
        path="../../data-sets/ml-latest/movies.csv",
        sep=",",
        encoding="UTF-8",
        header=True,
        quote='"',
        schema="movieId INT, title STRING, genres STRING",
    )
)

In [8]:
df_movies.show(n=5)
df_movies.printSchema()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



### As we can see we have ellipsis (...) at some places in title and genre column. By default df.show method will truncate long strings with ellipsis (...). We can disable this with truncate=False argument, as follows-

In [9]:
df_movies.show(n=15, truncate=False)
df_movies.printSchema()

+-------+----------------------------------+-------------------------------------------+
|movieId|title                             |genres                                     |
+-------+----------------------------------+-------------------------------------------+
|1      |Toy Story (1995)                  |Adventure|Animation|Children|Comedy|Fantasy|
|2      |Jumanji (1995)                    |Adventure|Children|Fantasy                 |
|3      |Grumpier Old Men (1995)           |Comedy|Romance                             |
|4      |Waiting to Exhale (1995)          |Comedy|Drama|Romance                       |
|5      |Father of the Bride Part II (1995)|Comedy                                     |
|6      |Heat (1995)                       |Action|Crime|Thriller                      |
|7      |Sabrina (1995)                    |Comedy|Romance                             |
|8      |Tom and Huck (1995)               |Adventure|Children                         |
|9      |Sudden Death

### Now we are going to work with filterring the data aka: Dicing and Slicing

In [10]:
# Let's filter on genre with value "Action" in movies dataframe
df_movies.where("genres" == "Action").show() # This is going to fail

TypeError: condition should be string or Column

### As you can see, we are unable to perform this action, as we require th condition to have Column object instead of string. We can fix this by using the SQL expression instead of the pythonic way of representing condition

In [82]:
df_movies.where('genres = "Action"').show() # Notice the quotes and single equals to "="

+-------+--------------------+------+
|movieId|               title|genres|
+-------+--------------------+------+
|      9| Sudden Death (1995)|Action|
|     71|    Fair Game (1995)|Action|
|    204|Under Siege 2: Da...|Action|
|    251|  Hunted, The (1995)|Action|
|    667|Bloodsport 2 (a.k...|Action|
|    980|Yes, Madam (a.k.a...|Action|
|   1102|American Strays (...|Action|
|   1110| Bird of Prey (1996)|Action|
|   1170|Best of the Best ...|Action|
|   1424|       Inside (1996)|Action|
|   1434|Stranger, The (1994)|Action|
|   1497|  Double Team (1997)|Action|
|   1599|        Steel (1997)|Action|
|   2196|    Knock Off (1998)|Action|
|   2258|  Master, The (1984)|Action|
|   2534|    Avalanche (1978)|Action|
|   2756|Wanted: Dead or A...|Action|
|   2817|Aces: Iron Eagle ...|Action|
|   2965|Omega Code, The (...|Action|
|   3283|Minnie and Moskow...|Action|
+-------+--------------------+------+
only showing top 20 rows



### As you can see, we can use proper SQL expression for filterring, and it works

### Another way to do the same thing is to convert the string "genre" to spark column, as follows-

In [83]:
df_movies.where(f.col("genres") == "Action").show(n=5, truncate=False)

+-------+-----------------------------------------------------------+------+
|movieId|title                                                      |genres|
+-------+-----------------------------------------------------------+------+
|9      |Sudden Death (1995)                                        |Action|
|71     |Fair Game (1995)                                           |Action|
|204    |Under Siege 2: Dark Territory (1995)                       |Action|
|251    |Hunted, The (1995)                                         |Action|
|667    |Bloodsport 2 (a.k.a. Bloodsport II: The Next Kumite) (1996)|Action|
+-------+-----------------------------------------------------------+------+
only showing top 5 rows



### I prefer the later, as it's more pythonic

### As we can see the genre column has values seperated by pipe "|" symbol. I would like to have it as an array instead of a single string. For doing that I will create a new column called genre_array, and use f.split to apply transformation on genre column to split on the symbol "|"

In [84]:
df_movies_with_genre = (
    df_movies
    .withColumn("genres_array", f.split("genres", "|"))
)

df_movies_with_genre.show(n=5)
df_movies_with_genre.printSchema()

+-------+--------------------+--------------------+--------------------+
|movieId|               title|              genres|        genres_array|
+-------+--------------------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|[A, d, v, e, n, t...|
|      2|      Jumanji (1995)|Adventure|Childre...|[A, d, v, e, n, t...|
|      3|Grumpier Old Men ...|      Comedy|Romance|[C, o, m, e, d, y...|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|[C, o, m, e, d, y...|
|      5|Father of the Bri...|              Comedy|[C, o, m, e, d, y, ]|
+-------+--------------------+--------------------+--------------------+
only showing top 5 rows

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- genres_array: array (nullable = true)
 |    |-- element: string (containsNull = true)



### As we can see, it didn't work as expected. This happened because the second argument of f.split is actually a regex. And in the world of regex, "|" character is a special symbol, hence it needs to be excaped.

In [85]:
df_movies_with_genre = (
    df_movies
    .withColumn("genres_array", f.split("genres", "\|")) # notice the escape symbol "\"
)

df_movies_with_genre.show(n=5, truncate=False)
df_movies_with_genre.printSchema()

+-------+----------------------------------+-------------------------------------------+-------------------------------------------------+
|movieId|title                             |genres                                     |genres_array                                     |
+-------+----------------------------------+-------------------------------------------+-------------------------------------------------+
|1      |Toy Story (1995)                  |Adventure|Animation|Children|Comedy|Fantasy|[Adventure, Animation, Children, Comedy, Fantasy]|
|2      |Jumanji (1995)                    |Adventure|Children|Fantasy                 |[Adventure, Children, Fantasy]                   |
|3      |Grumpier Old Men (1995)           |Comedy|Romance                             |[Comedy, Romance]                                |
|4      |Waiting to Exhale (1995)          |Comedy|Drama|Romance                       |[Comedy, Drama, Romance]                         |
|5      |Father of the Brid

### Now for we want to create a single row for every movie, genre pair. In order to that, we will need to "explode" the genres_array column using f.explode

In [86]:
df_movies_with_genre = (
    df_movies
    .withColumn("genres_array", f.split("genres", "\|")) # notice the escape symbol "\"
    .withColumn("genre", f.explode("genres_array"))
)

df_movies_with_genre.show(n=15, truncate=False)
df_movies_with_genre.printSchema()

+-------+----------------------------------+-------------------------------------------+-------------------------------------------------+---------+
|movieId|title                             |genres                                     |genres_array                                     |genre    |
+-------+----------------------------------+-------------------------------------------+-------------------------------------------------+---------+
|1      |Toy Story (1995)                  |Adventure|Animation|Children|Comedy|Fantasy|[Adventure, Animation, Children, Comedy, Fantasy]|Adventure|
|1      |Toy Story (1995)                  |Adventure|Animation|Children|Comedy|Fantasy|[Adventure, Animation, Children, Comedy, Fantasy]|Animation|
|1      |Toy Story (1995)                  |Adventure|Animation|Children|Comedy|Fantasy|[Adventure, Animation, Children, Comedy, Fantasy]|Children |
|1      |Toy Story (1995)                  |Adventure|Animation|Children|Comedy|Fantasy|[Adventure, Animat

### As we can see, each movie is "exploded" across every genre type. But it's hard to see, so we will select the rows we are interested in before showing

In [87]:
df_movies_with_genre = (
    df_movies
    .withColumn("genres_array", f.split("genres", "\|")) # notice the escape symbol "\"
    .withColumn("genre", f.explode("genres_array"))
    .select("movieId", "title", "genre")
)

df_movies_with_genre.show(n=15, truncate=False)
df_movies_with_genre.printSchema()

+-------+----------------------------------+---------+
|movieId|title                             |genre    |
+-------+----------------------------------+---------+
|1      |Toy Story (1995)                  |Adventure|
|1      |Toy Story (1995)                  |Animation|
|1      |Toy Story (1995)                  |Children |
|1      |Toy Story (1995)                  |Comedy   |
|1      |Toy Story (1995)                  |Fantasy  |
|2      |Jumanji (1995)                    |Adventure|
|2      |Jumanji (1995)                    |Children |
|2      |Jumanji (1995)                    |Fantasy  |
|3      |Grumpier Old Men (1995)           |Comedy   |
|3      |Grumpier Old Men (1995)           |Romance  |
|4      |Waiting to Exhale (1995)          |Comedy   |
|4      |Waiting to Exhale (1995)          |Drama    |
|4      |Waiting to Exhale (1995)          |Romance  |
|5      |Father of the Bride Part II (1995)|Comedy   |
|6      |Heat (1995)                       |Action   |
+-------+-

### Now it's much better to visualize

### Now let's list out all the distinct genre using .distinct

In [88]:
available_genres = df_movies_with_genre.select("genre").distinct()
available_genres.show(truncate=False)

+------------------+
|genre             |
+------------------+
|Crime             |
|Romance           |
|Thriller          |
|Adventure         |
|Drama             |
|War               |
|Documentary       |
|Fantasy           |
|Mystery           |
|Musical           |
|Animation         |
|Film-Noir         |
|(no genres listed)|
|IMAX              |
|Horror            |
|Western           |
|Comedy            |
|Children          |
|Action            |
|Sci-Fi            |
+------------------+



### We can see all the distinct genres, there is a value- (no genres listed). Let's figure out all the movies with (no genres listed)

In [89]:
df_movies_with_no_genres = (
    df_movies
    .filter(f.col("genres") == "(no genres listed)")
)
print(f"Total: {df_movies_with_no_genres.count()} movie(s) does not have genres")
df_movies_with_no_genres.show(truncate=False)

Total: 4266 movie(s) does not have genres
+-------+-----------------------------------------------+------------------+
|movieId|title                                          |genres            |
+-------+-----------------------------------------------+------------------+
|83773  |Away with Words (San tiao ren) (1999)          |(no genres listed)|
|83829  |Scorpio Rising (1964)                          |(no genres listed)|
|84768  |Glitterbug (1994)                              |(no genres listed)|
|86493  |Age of the Earth, The (A Idade da Terra) (1980)|(no genres listed)|
|87061  |Trails (Veredas) (1978)                        |(no genres listed)|
|91246  |Milky Way (Tejút) (2007)                       |(no genres listed)|
|92435  |Dancing Hawk, The (Tanczacy jastrzab) (1978)   |(no genres listed)|
|92641  |Warsaw Bridge (Pont de Varsòvia) (1990)        |(no genres listed)|
|94431  |Ella Lola, a la Trilby (1898)                  |(no genres listed)|
|94657  |Turkish Dance, Ella Lola 

In [90]:
spark.stop()